In [1]:
%%time

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

CPU times: user 1.13 s, sys: 163 ms, total: 1.29 s
Wall time: 1.71 s


In [2]:
%%time

train = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')

CPU times: user 146 ms, sys: 31 ms, total: 177 ms
Wall time: 246 ms


In [3]:
%%time

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

In [4]:
%%time

# check missing values

missing_values = train.isnull().sum()
missing_values

CPU times: user 23.5 ms, sys: 1.08 ms, total: 24.5 ms
Wall time: 23.6 ms


id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64

In [5]:
%%time

import h2o
from h2o.automl import H2OAutoML

# Initialize the H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6gjgtm7x
  JVM stdout: /tmp/tmp6gjgtm7x/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6gjgtm7x/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_unknownUser_k0i3a3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.250 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


CPU times: user 366 ms, sys: 82.7 ms, total: 449 ms
Wall time: 7.55 s


In [6]:
%%time

# Converting training data to H2O Frame
train_h2o = h2o.H2OFrame(train)
test_h2o = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 1.05 s, sys: 55.4 ms, total: 1.11 s
Wall time: 4.73 s


In [7]:
%%time

# Define the target and features
target = 'loan_status'
features = [col for col in train_h2o.columns if col not in ['id', target]]

CPU times: user 17 µs, sys: 2 µs, total: 19 µs
Wall time: 24.1 µs


In [8]:
%%time

# Convert train categorical columns are set as factors
train_h2o['person_home_ownership'] = train_h2o['person_home_ownership'].asfactor()
train_h2o['loan_intent'] = train_h2o['loan_intent'].asfactor()
train_h2o['loan_grade'] = train_h2o['loan_grade'].asfactor()
train_h2o['cb_person_default_on_file'] = train_h2o['cb_person_default_on_file'].asfactor()
train_h2o['loan_status'] = train_h2o['loan_status'].asfactor()  # Target column

# Convert test categorical columns to factors
test_h2o['person_home_ownership'] = test_h2o['person_home_ownership'].asfactor()
test_h2o['loan_intent'] = test_h2o['loan_intent'].asfactor()
test_h2o['loan_grade'] = test_h2o['loan_grade'].asfactor()
test_h2o['cb_person_default_on_file'] = test_h2o['cb_person_default_on_file'].asfactor()

CPU times: user 1.31 ms, sys: 0 ns, total: 1.31 ms
Wall time: 1.31 ms


In [9]:
%%time

# Split the data into training and validation sets
train, valid = train_h2o.split_frame(ratios=[0.8], seed=42)

CPU times: user 274 ms, sys: 1.93 ms, total: 276 ms
Wall time: 821 ms


In [10]:
%%time

# Initializing H2O AutoML
aml = H2OAutoML(max_runtime_secs=1000, seed=42, nfolds=5, sort_metric='AUC')

CPU times: user 3.84 ms, sys: 133 µs, total: 3.97 ms
Wall time: 33.8 ms


In [11]:
%%time

# Train the AutoML model
aml.train(x=features, y=target, training_frame=train, validation_frame=valid)

AutoML progress: |
00:34:13.19: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 7.19 s, sys: 622 ms, total: 7.81 s
Wall time: 16min 41s


key,value
Stacking strategy,cross_validation
Number of base models (used / total),13/42
# GBM base models (used / total),1/16
# XGBoost base models (used / total),11/20
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),0/3
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [12]:
%%time 

# View the leaderboard
lb = aml.leaderboard
print(lb)

model_id                                                     auc    logloss     aucpr    mean_per_class_error      rmse        mse
StackedEnsemble_AllModels_3_AutoML_1_20241007_03412     0.95479    0.149991  0.873398                0.137324  0.199284  0.0397141
StackedEnsemble_AllModels_2_AutoML_1_20241007_03412     0.953949   0.150993  0.872062                0.138871  0.199722  0.039889
StackedEnsemble_BestOfFamily_4_AutoML_1_20241007_03412  0.953726   0.151557  0.870955                0.137485  0.200229  0.0400917
StackedEnsemble_BestOfFamily_3_AutoML_1_20241007_03412  0.953704   0.151525  0.871069                0.136961  0.200213  0.0400852
XGBoost_3_AutoML_1_20241007_03412                       0.952726   0.153069  0.869438                0.138371  0.20133   0.0405337
XGBoost_grid_1_AutoML_1_20241007_03412_model_14         0.95261    0.152867  0.86916                 0.133927  0.200867  0.0403474
StackedEnsemble_AllModels_1_AutoML_1_20241007_03412     0.952447   0.152841  0.86842

In [13]:
%%time

# Make predictions on the test set
preds = aml.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
CPU times: user 282 ms, sys: 6.57 ms, total: 288 ms
Wall time: 3.53 s


In [18]:
%%time

# Extract the predicted labels
predicted_labels = preds['predict'].as_data_frame().values.flatten()

CPU times: user 33.4 ms, sys: 2.78 ms, total: 36.2 ms
Wall time: 81.6 ms


/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [19]:
%%time

# Create submission DataFrame
submission = sample_submission.copy()
submission['loan_status'] = predicted_labels

CPU times: user 1.88 ms, sys: 0 ns, total: 1.88 ms
Wall time: 1.23 ms


In [20]:
%%time

# Save the submission file
submission_file_path = 'submission.csv'
submission.to_csv(submission_file_path, index=False)

CPU times: user 41.2 ms, sys: 4.92 ms, total: 46.1 ms
Wall time: 45 ms


In [21]:
%%time

submission.head()

CPU times: user 232 µs, sys: 0 ns, total: 232 µs
Wall time: 235 µs


,id,loan_status
0,58645,1
1,58646,0
2,58647,1
3,58648,0
4,58649,0
